# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [22]:
import os

import chromadb
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

In [23]:
load_dotenv(".env")
os.environ["OPENAI_BASE_URL"] = "https://openai.vocareum.com/v1"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert OPENAI_API_KEY and OPENAI_API_KEY.startswith("voc-"), "Need a voc- OPENAI_API_KEY"
assert TAVILY_API_KEY, "Missing TAVILY_API_KEY"

openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=os.getenv("OPENAI_BASE_URL"))
tavily = TavilyClient(TAVILY_API_KEY)

In [24]:
from lib.voc_embedding import VocareumEmbeddingFunction

embedding_fn = VocareumEmbeddingFunction("text-embedding-3-small")
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay", embedding_function=embedding_fn)

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [25]:
def retrieve_game(q, k=3):
    r = collection.query(
        query_texts=[q], n_results=k,
        include=["metadatas","distances"]
    )
    out = []
    for i, m in enumerate(r["metadatas"][0]):
        out.append({
            "name": m.get("Name"),
            "platform": m.get("Platform"),
            "year": m.get("YearOfRelease"),
            "publisher": m.get("Publisher"),
            "score": round(1 - r["distances"][0][i], 4)
        })
    return out

retrieve_game("What platform was Gran Turismo launched on?", 2)


[{'name': 'Gran Turismo',
  'platform': 'PlayStation 1',
  'year': 1997,
  'publisher': 'Sony Computer Entertainment',
  'score': 0.2703},
 {'name': 'Gran Turismo 5',
  'platform': 'PlayStation 3',
  'year': 2010,
  'publisher': 'Sony Computer Entertainment',
  'score': 0.1987}]

#### Evaluate Retrieval Tool

In [26]:
import json

def evaluate_retrieval(question, retrieved_docs):
    docs = "\n".join([f"- {d.get('name')} | {d.get('platform')} | {d.get('year')}"
                      for d in retrieved_docs[:3]])
    ask = (
        "Are these docs enough to answer the question?\n"
        "Reply ONLY as JSON with keys: useful (true/false), description (short).\n\n"
        f"Question: {question}\nDocs:\n{docs}"
    )
    r = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type":"json_object"},
        messages=[{"role":"system","content":"Be brief. JSON only."},
                  {"role":"user","content": ask}]
    )
    return json.loads(r.choices[0].message.content)

# quick test
hits = retrieve_game("What platform was Gran Turismo launched on?", k=3)
evaluate_retrieval("What platform was Gran Turismo launched on?", hits)

{'useful': True,
 'description': 'The docs mention Gran Turismo was launched on PlayStation 1 in 1997.'}

#### Game Web Search Tool

In [27]:
def game_web_search(question, max_results=5):
    r = tavily.search(
        query=question,
        max_results=max_results,
        include_answer=True
    )
    answer = r.get("answer")
    results = [{"title": x.get("title"), "url": x.get("url")} for x in r.get("results", [])]
    return {"answer": answer, "results": results}


w = game_web_search("When were Pokémon Gold and Silver released?", max_results=5)
print("Answer:", w["answer"])
print("Results shown:", len(w["results"]))


Answer: Pokémon Gold and Silver were released in Japan on November 21, 1999, and in North America on October 15, 2000.
Results shown: 5


In [28]:
def tool(fn):
    fn._is_tool = True
    fn.tool_name = fn.__name__
    return fn

@tool
def retrieve_game_tool(query, k=5):
    return retrieve_game(query, k)

@tool
def evaluate_retrieval_tool(question, retrieved_docs):
    return evaluate_retrieval(question, retrieved_docs)

@tool
def game_web_search_tool(question, max_results=5):
    return game_web_search(question, max_results)


In [29]:
def udaplay(q):
    hits  = retrieve_game(q, k=5)
    judge = evaluate_retrieval(q, hits)

    if judge.get("useful"):
        top = hits[0] if hits else {}
        ql = q.lower()
        if "platform" in ql:
            ans = f"{top.get('name')} was launched on {top.get('platform')}."
        elif "publisher" in ql:
            ans = f"The publisher of {top.get('name')} is {top.get('publisher')}."
        else:
            ans = f"{top.get('name')} ({top.get('platform')}) was released in {top.get('year')}."
        return {"mode":"local","answer":ans,"sources":hits,"eval":judge}

    web = game_web_search(q, max_results=5)
    return {"mode":"web","answer":web.get("answer") or "See sources.","sources":web.get("results",[]),"eval":judge}


### Agent

In [ ]:
from enum import Enum

class State(Enum):
    RETRIEVE = "RETRIEVE"
    EVALUATE = "EVALUATE"
    WEB      = "WEB"
    SUMMARIZE= "SUMMARIZE"
    END      = "END"

class MiniAgent:
    def __init__(self, tools):
        self.tools = {t.tool_name: t for t in tools}
        self.history = []          
        self.last_game_name = ""   

    def _with_context(self, question: str) -> str:
        ql = question.lower()
        if self.last_game_name and any(w in ql for w in [" it", " that", " this", "that game", "this game"]):
            return f"{question} (context: about {self.last_game_name})"
        return question

    def run(self, question: str):
        state = State.RETRIEVE
        trace = [("USER", question)]

        hits = None
        report = None
        web = None

        question_for_tools = self._with_context(question)

        while state != State.END:
            if state == State.RETRIEVE:
                trace.append(("ASSISTANT", "None 🛠 Tool Call: retrieve_game"))
                hits = self.tools["retrieve_game_tool"](question_for_tools, k=5)
                trace.append(("TOOL", hits))
                state = State.EVALUATE

            elif state == State.EVALUATE:
                trace.append(("ASSISTANT", "None 🛠 Tool Call: evaluate_retrieval"))
                report = self.tools["evaluate_retrieval_tool"](question, hits or [])
                trace.append(("TOOL", report))
                state = State.SUMMARIZE if report.get("useful") else State.WEB

            elif state == State.WEB:
                trace.append(("ASSISTANT", "None 🛠 Tool Call: game_web_search"))
                web = self.tools["game_web_search_tool"](question, max_results=5)
                trace.append(("TOOL", web))
                state = State.SUMMARIZE

            elif state == State.SUMMARIZE:
                if report and report.get("useful") and hits:
                    top = hits[0]
                    ql = question.lower()
                    if "platform" in ql:
                        answer = f"{top['name']} was launched on {top['platform']}."
                    elif "publisher" in ql:
                        answer = f"The publisher of {top['name']} is {top['publisher']}."
                    else:
                        answer = f"{top['name']} ({top['platform']}) was released in {top['year']}."
                    mode = "local"
                    sources = hits[:2]
                    self.last_game_name = top.get("name") or self.last_game_name
                else:
                    answer = (web or {}).get("answer") or "I looked this up on the web. See sources."
                    mode = "web"
                    sources = (web or {}).get("results", [])[:2]
                state = State.END

        for role, content in trace:
            if role == "USER":
                print(f"🧑‍💻 USER: {content}")
            elif role == "ASSISTANT":
                print(f"🎮 ASSISTANT: {content}")
            else:
                print(f"🧰 TOOL: {content}")

        print("\n🎮 ASSISTANT: Summarised Answer:")
        print(answer)

        print("\nCitations:")
        if mode == "local":
            for s in sources:
                print(f"- {s['name']} ({s['platform']}, {s.get('year')})")
        else:
            for s in sources:
                print(f"- {s.get('title')} — {s.get('url')}")

        print("\nReport:")
        if mode == "local" and isinstance(report, dict):
            print(report.get("description", "Used local docs."))
        else:
            print("Fell back to web search.")

        self.history.append({
            "q": question, "mode": mode, "answer": answer,
            "sources": sources, "trace": trace
        })
        return {"answer": answer, "mode": mode, "sources": sources, "trace": trace}

In [31]:
import json

agent = MiniAgent([retrieve_game_tool, evaluate_retrieval_tool, game_web_search_tool])

def run_structured(q):
    out = agent.run(q)
    payload = {
        "question": q,
        "answer": out["answer"],
        "mode": out["mode"],          
        "sources": out["sources"],  
        "history_len": len(agent.history)
    }
    print("\nJSON:")
    print(json.dumps(payload, indent=2))
    return payload

for q in [
    "When were Pokémon Gold and Silver released?",
    "Which was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]:
    _ = run_structured(q)

🧑‍💻 USER: When were Pokémon Gold and Silver released?
🎮 ASSISTANT: None 🛠 Tool Call: retrieve_game
🧰 TOOL: [{'name': 'Pokémon Gold and Silver', 'platform': 'Game Boy Color', 'year': 1999, 'publisher': 'Nintendo', 'score': 0.3122}, {'name': 'Pokémon Ruby and Sapphire', 'platform': 'Game Boy Advance', 'year': 2002, 'publisher': 'Nintendo', 'score': -0.0213}, {'name': 'Wii Sports', 'platform': 'Wii', 'year': 2006, 'publisher': 'Nintendo', 'score': -0.5319}, {'name': 'Mario Kart 8 Deluxe', 'platform': 'Nintendo Switch', 'year': 2017, 'publisher': 'Nintendo', 'score': -0.5375}, {'name': 'Super Mario World', 'platform': 'Super Nintendo Entertainment System (SNES)', 'year': 1990, 'publisher': 'Nintendo', 'score': -0.5441}]
🎮 ASSISTANT: None 🛠 Tool Call: evaluate_retrieval
🧰 TOOL: {'useful': True, 'description': 'The docs contain the release year for Pokémon Gold and Silver.'}

🎮 ASSISTANT: Summarised Answer:
Pokémon Gold and Silver (Game Boy Color) was released in 1999.

Citations:
- Pokémon 

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes